In [ ]:
import h5py
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.utils.data as data
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load datasets
photon_file = "D:/Photon_Electron_Detection/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5"
electron_file = "D:/Photon_Electron_Detection/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5"

def load_hdf5_data(file):
    with h5py.File(file, "r") as f:
        images = np.array(f["X"])
        labels = np.array(f["y"])
    return images, labels

# Load data
photon_images, photon_labels = load_hdf5_data(photon_file)
electron_images, electron_labels = load_hdf5_data(electron_file)

# Assign labels (Photons = 0, Electrons = 1)
photon_labels[:] = 0
electron_labels[:] = 1

# Combine both datasets
X = np.concatenate((photon_images, electron_images), axis=0)
y = np.concatenate((photon_labels, electron_labels), axis=0)

# Normalize the images
X = X.astype(np.float32) / 255.0

# Reshape for PyTorch (Batch, Channels, Height, Width)
X = np.expand_dims(X, axis=1)  # Add channel dimension

# Train-Test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Convert to PyTorch tensors
X_train, y_train = torch.tensor(X_train), torch.tensor(y_train)
X_test, y_test = torch.tensor(X_test), torch.tensor(y_test)

# Create PyTorch datasets
train_dataset = data.TensorDataset(X_train, y_train)
test_dataset = data.TensorDataset(X_test, y_test)

# Data loaders
train_loader = data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define ResNet-15 Model
class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return torch.relu(out)

class ResNet15(nn.Module):
    def __init__(self):
        super(ResNet15, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = ResNetBlock(64, 128, stride=2)
        self.layer2 = ResNetBlock(128, 256, stride=2)
        self.layer3 = ResNetBlock(256, 512, stride=2)
        self.fc = nn.Linear(512 * 4 * 4, 1)  # Adjust based on feature map size

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = torch.flatten(out, 1)
        out = torch.sigmoid(self.fc(out))
        return out

# Initialize model
model = ResNet15().to(device)

# Define Loss and Optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.float().to(device).unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Train model
train_model(model, train_loader, epochs=10)

# Save model weights
torch.save(model.state_dict(), "resnet15_model.pth")

# Evaluate Model
model.eval()
y_true = []
y_scores = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        y_true.extend(labels.cpu().numpy())
        y_scores.extend(outputs.cpu().numpy())

# Compute ROC-AUC Score
roc_auc = roc_auc_score(y_true, y_scores)
print(f"ROC-AUC Score: {roc_auc:.4f}")

# Save results
with open("results.txt", "w") as f:
    f.write(f"ROC-AUC Score: {roc_auc:.4f}\n")

print("Training and evaluation complete. Model and results saved.")
